In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# import os
# os.chdir("/content/gdrive/MyDrive/Colab Notebooks/RAGLangChain")

In [ ]:
# !pip install -q langchain
# !pip install -q chromadb
# !pip install -q langchain_community
# !pip install -q langchain_chroma
# !pip install -U -q langchain-huggingface
# !pip install -q openai
# !pip install -q tqdm

## Embeding ThuVienPhapLuat


ModuleNotFoundError: No module named 'langchain'

In [9]:
import os
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
import csv
import re
import pandas as pd

ModuleNotFoundError: No module named 'langchain'

In [13]:
file_path = '../data/So-huu-tri-tue-processed.csv'
df = pd.read_csv(file_path)
df.head()

,link,loai_van_ban,ngay_ban_hanh,ngay_cong_bao,ngay_hieu_luc,nguoi_ky,noi_ban_hanh,so_cong_bao,so_hieu,title,van_ban_duoc_dan,noi_dung_day_du
0,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định,20/01/2017,Đang cập nhật,Đã biết,Vũ Đức Đam,Thủ tướng Chính phủ,Đang cập nhật,88/QĐ-TTg,Quyết định 88/QĐ-TTg năm 2017 phê duyệt Đề án ...,{'Nghị quyết 102/NQ-CP': 'https://thuvienphapl...,THỦ TƯỚNG CHÍNH PHỦ ----------: Số: 88/QĐ-TT...
1,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định,19/01/2017,Đang cập nhật,Đã biết,Huỳnh Thanh Điền,Tỉnh Nghệ An,Đang cập nhật,320/QĐ-UBND,Quyết định 320/QĐ-UBND năm 2017 về phê duyệt C...,{},ỦY BAN NHÂN DÂN TỈNH NGHỆ AN -------: Số: 32...
2,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định,13/02/2015,Đang cập nhật,Đã biết,Phạm Thế Dũng,Tỉnh Gia Lai,Đang cập nhật,160/QĐ-UBND,Quyết định 160/QĐ-UBND phê duyệt danh mục nhiệ...,{},ỦY BAN NHÂN DÂN TỈNH GIA LAI -------: Số: 160...
3,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định,05/02/2015,Đang cập nhật,Đã biết,Nguyễn Xuân Hồng,Cục Bảo vệ thực vật,Đang cập nhật,238/QĐ-BVTV,Quyết định 238/QĐ-BVTV năm 2015 công nhận “Quy...,{'Quyết định 86/2008/QĐ-BNN': 'https://thuvien...,BỘ NÔNG NGHIỆP VÀ PHÁT TRIỂN NÔNG THÔN CỤC BẢ...
4,https://thuvienphapluat.vn/van-ban/So-huu-tri-...,Quyết định,14/06/2012,Đang cập nhật,Đã biết,Lý Hải Hầu,Tỉnh Cao Bằng,Đang cập nhật,739/QĐ-UBND,Quyết định 739/QĐ-UBND năm 2012 sửa đổi Quyết ...,{},ỦY BAN NHÂN DÂN TỈNH CAO BẰNG --------: Số:...


In [ ]:
def update_metadata(doc: Document):
    # Lấy nội dung văn bản
    text = doc.page_content

    metadata = {}

    metadata['link'] = re.search(r'link:\s*(https?://\S+)', text)
    metadata['link'] = metadata['link'].group(1) if metadata['link'] else "N/A"

    metadata['loai_van_ban'] = re.search(r'loai_van_ban:\s*(\S.+)', text)
    metadata['loai_van_ban'] = metadata['loai_van_ban'].group(1) if metadata['loai_van_ban'] else "N/A"

    metadata['ngay_ban_hanh'] = re.search(r'ngay_ban_hanh:\s*(\S.+)', text)
    metadata['ngay_ban_hanh'] = metadata['ngay_ban_hanh'].group(1) if metadata['ngay_ban_hanh'] else "N/A"

    metadata['noi_ban_hanh'] = re.search(r'noi_ban_hanh:\s*(\S.+)', text)
    metadata['noi_ban_hanh'] = metadata['noi_ban_hanh'].group(1) if metadata['noi_ban_hanh'] else "N/A"

    metadata['so_hieu'] = re.search(r'so_hieu:\s*(\S+)', text)
    metadata['so_hieu'] = metadata['so_hieu'].group(1) if metadata['so_hieu'] else "N/A"

    metadata['title'] = re.search(r'title:\s*(\S.+)', text)
    metadata['title'] = metadata['title'].group(1) if metadata['title'] else "N/A"

    metadata['van_ban_duoc_dan'] = re.search(r'van_ban_duoc_dan:\s*(\{.+\})', text)
    metadata['van_ban_duoc_dan'] = metadata['van_ban_duoc_dan'].group(1) if metadata['van_ban_duoc_dan'] else "N/A"

    # Cập nhật metadata vào document
    doc.metadata.update(metadata)

    return doc

In [ ]:
def load_data(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")

    csv.field_size_limit(10**6)
    loader = CSVLoader(file_path, encoding="utf-8")

    try:
        documents = loader.load()
    except Exception as e:
        raise ValueError(f"Failed to load CSV file '{file_path}': {e}")

    updated_documents = [update_metadata(doc) for doc in tqdm(documents)]

    return updated_documents


In [ ]:
documents = load_data(file_path)

In [ ]:
print(f'Tổng số tài liệu{len(documents)}')
print(documents[0].metadata)

In [ ]:
def create_chroma_db(documents, persist_dir, batch_size=1000):
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)

    print(f"Number of chunks: {len(docs)}")

    if not os.path.exists(persist_dir):
        os.makedirs(persist_dir)

    vector_db = None
    for i in tqdm(range(0, len(docs), batch_size)):
        batch = docs[i:i+batch_size]

        if vector_db is None:
            vector_db = Chroma.from_documents(documents=batch, embedding=embeddings, persist_directory=persist_dir)
        else:
            vector_db.add_documents(batch)

        print(f"Persisted batch {i//batch_size + 1}")

    print(f"Database persisted to {persist_dir}")

    return vector_db


In [16]:
def load_chroma_db(persist_dir):
    if not os.path.exists(persist_dir):
        raise FileNotFoundError(f"The directory '{persist_dir}' does not exist.")

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )

    # Load the Chroma vector store
    vector_db = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    return vector_db

In [ ]:
# vector_db = create_chroma_db(documents, "vector_db")

In [34]:
vector_db = load_chroma_db("../vector_db")

## Chatbot

In [39]:
import os

from dotenv import load_dotenv
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_openai import ChatOpenAI
from sentence_transformers import CrossEncoder

In [58]:
# Load biến môi trường từ file .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [59]:
# Khởi tạo mô hình LLM với cấu hình
llm = ChatOpenAI(
    temperature=0,
    streaming=True,
    model="gpt-4o-mini",
    openai_api_key=openai_api_key,
)

In [60]:
# Định nghĩa mẫu prompt
system = """You are a Intellectual property Lawer . Your name is iLaw. You are here to help people with their legal questions.
Here is the context you should refer to:
{context}
Your response must be in Vietnamese. 
If there is no relevant documentation, reply no data.
"""
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "Question: {question}")]
)

# Khởi tạo chuỗi RAG với prompt_template
rag_chain = (
    RunnableMap(
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    )
    | prompt_template
    | llm
    | StrOutputParser()
)

In [70]:
class EnhancedEnsembleRetriever:
    def __init__(self, retrievers, weights, reranker_model=None):
        self.retrievers = retrievers
        self.weights = weights
        self.reranker = CrossEncoder(reranker_model) if reranker_model else None

    def invoke(self, query):
        # 1. Lấy kết quả từ từng retriever
        all_results = []
        for retriever, weight in zip(self.retrievers, self.weights):
            results = retriever.get_relevant_documents(query)
            for doc in results:
                doc.metadata['score'] = doc.metadata.get('score', 1.0) * weight
            all_results.extend(results)

        # 2. Loại bỏ trùng lặp (nếu cần) theo nội dung
        unique_results = []
        seen_content = set()

        for doc in all_results:
            content = doc.page_content
            if content not in seen_content:
                unique_results.append(doc)
                seen_content.add(content)
        

        # 3. Rerank nếu có mô hình reranker
        if self.reranker:
            query_doc_pairs = [(query, doc.page_content) for doc in unique_results]
            scores = self.reranker.predict(query_doc_pairs)
            reranked_results = sorted(
                zip(unique_results, scores),
                key=lambda x: x[1],  # Sắp xếp theo điểm số
                reverse=True
            )
            return [doc for doc, score in reranked_results]

        # 4. Nếu không rerank, trả về kết quả theo trọng số
        return sorted(unique_results, key=lambda x: x.metadata['score'], reverse=True)

In [71]:
# Hàm tạo retriever từ Chroma và BM25
def create_retriever(vector_db, query, k=4):

    # Tạo BM25 retriever
    chroma_retriever = vector_db.as_retriever(search_type='similarity', search_kwargs={'k': 4})

    documents_bm25 = [
        Document(page_content=doc.page_content, metadata=doc.metadata)
        for doc in vector_db.similarity_search("", k=300)
    ]

    bm25_retriever = BM25Retriever.from_documents(documents_bm25)
    bm25_retriever.k = 4

    ensemble_retriever = EnhancedEnsembleRetriever(
        retrievers=[chroma_retriever, bm25_retriever],
        weights=[0.7, 0.3],
        reranker_model='cross-encoder/ms-marco-MiniLM-L-12-v2'
    )

    return ensemble_retriever

In [63]:
# Hàm lấy tài liệu phù hợp từ retriever
def retrieve_documents(ensemble_retriever, query, top_k=4):
    docs = ensemble_retriever.invoke(query=query)
    return docs[:top_k]
# Hàm định dạng tài liệu thành chuỗi
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

def remove_json_formatting(input_text):
    # Loại bỏ dấu ```json và ``` nếu chúng có trong input_text
    cleaned_text = input_text.strip("```json").strip("```").strip()
    return cleaned_text


In [72]:
# Hàm gọi chuỗi RAG và sinh câu trả lời
def generate_answer(vector_db, question, top_k=4):
    ensemble_retriever = create_retriever( vector_db, question, k=top_k)
    docs = retrieve_documents(ensemble_retriever, question, top_k=top_k)
    links =[]
    titles = []
    # Lấy những thông tin cần thiết từ metadata của tài liệu: link, title
    for doc in docs:
        links.append(doc.metadata['link'])
        titles.append(doc.metadata['title'])
        
    formatted_docs = format_docs(docs)
    output = rag_chain.invoke({"context": formatted_docs, "question": question})
    # output = remove_json_formatting(output)
    # print(formatted_docs)
    return output, links, titles

In [73]:
question = "Quy định về bản quyền trong luật sở hữu trí tuệ"

In [74]:
output, links, titles = generate_answer(vector_db, question, top_k=4)

In [75]:
links

['https://thuvienphapluat.vn/van-ban/So-huu-tri-tue/Thong-tu-lien-tich-02-2008-TTLT-TANDTC-VKSNDTC-BVHTT-DL-BKH-CN-BTP-huong-dan-ap-dung-quy-dinh-cua-phap-luat-giai-quyet-cac-tranh-chap-65887.aspx',
 'https://thuvienphapluat.vn/van-ban/So-huu-tri-tue/Thong-tu-lien-tich-02-2008-TTLT-TANDTC-VKSNDTC-BVHTT-DL-BKH-CN-BTP-huong-dan-ap-dung-quy-dinh-cua-phap-luat-giai-quyet-cac-tranh-chap-65887.aspx',
 'https://thuvienphapluat.vn/van-ban/So-huu-tri-tue/Thong-tu-lien-tich-02-2008-TTLT-TANDTC-VKSNDTC-BVHTT-DL-BKH-CN-BTP-huong-dan-ap-dung-quy-dinh-cua-phap-luat-giai-quyet-cac-tranh-chap-65887.aspx',
 'https://thuvienphapluat.vn/van-ban/So-huu-tri-tue/Thong-tu-lien-tich-02-2008-TTLT-TANDTC-VKSNDTC-BVHTT-DL-BKH-CN-BTP-huong-dan-ap-dung-quy-dinh-cua-phap-luat-giai-quyet-cac-tranh-chap-65887.aspx']

In [76]:
titles

['Thông tư liên tịch 02/2008/TTLT-TANDTC-VKSNDTC-BVHTT&DL-BKH&CN-BTP hướng dẫn giải quyết các tranh chấp về quyền sở hữu trí tuệ tại Toà án nhân dân do Tòa án nhân dân tối cao - Viện Kiểm sát nhân dân tối cao - Bộ Văn hóa, Thể thao và Du lịch - Bộ Khoa học và Công nghệ - Bộ Tư pháp ban hành',
 'Thông tư liên tịch 02/2008/TTLT-TANDTC-VKSNDTC-BVHTT&DL-BKH&CN-BTP hướng dẫn giải quyết các tranh chấp về quyền sở hữu trí tuệ tại Toà án nhân dân do Tòa án nhân dân tối cao - Viện Kiểm sát nhân dân tối cao - Bộ Văn hóa, Thể thao và Du lịch - Bộ Khoa học và Công nghệ - Bộ Tư pháp ban hành',
 'Thông tư liên tịch 02/2008/TTLT-TANDTC-VKSNDTC-BVHTT&DL-BKH&CN-BTP hướng dẫn giải quyết các tranh chấp về quyền sở hữu trí tuệ tại Toà án nhân dân do Tòa án nhân dân tối cao - Viện Kiểm sát nhân dân tối cao - Bộ Văn hóa, Thể thao và Du lịch - Bộ Khoa học và Công nghệ - Bộ Tư pháp ban hành',
 'Thông tư liên tịch 02/2008/TTLT-TANDTC-VKSNDTC-BVHTT&DL-BKH&CN-BTP hướng dẫn giải quyết các tranh chấp về quyền sở h

In [78]:
output

'Trong Luật sở hữu trí tuệ, quyền tài sản thuộc quyền tác giả được quy định tại Điều 20. Theo đó, quyền tài sản bao gồm các quyền như sao chép tác phẩm, cho phép tạo tác phẩm phái sinh, phân phối, nhập khẩu bản gốc và bản sao tác phẩm, truyền đạt tác phẩm đến công chúng, và cho thuê bản gốc hoặc bản sao chương trình máy tính. \n\nTuy nhiên, có sự khác biệt giữa quy định trong Luật sở hữu trí tuệ và Bộ luật dân sự năm 2005. Cụ thể, Bộ luật dân sự năm 2005 không quy định về quyền biểu diễn tác phẩm trước công chúng và cho thuê bản gốc hoặc bản sao tác phẩm điện ảnh, điều này có thể dẫn đến những tranh chấp trong việc áp dụng pháp luật.\n\nKhi giải quyết các tranh chấp liên quan đến quyền tác giả, Tòa án sẽ áp dụng quy định của Luật sở hữu trí tuệ, đặc biệt là trong trường hợp quyền tác giả vẫn còn trong thời hạn bảo hộ. Nếu quyền tác giả đã hết thời hạn bảo hộ, Tòa án sẽ không thụ lý đơn khởi kiện để giải quyết. \n\nNgoài ra, quyền sở hữu công nghiệp cũng được xác lập theo các quy định c